In [ ]:
import gen3schemadev as g

In [ ]:
lipid_schema = g.utils.load_yaml("../tests/gen3_schema/testing/yaml_pass/demographic.yaml")
rule_val = g.validators.rule_validator.RuleValidator(lipid_schema)
rule_val.validate()

In [ ]:
sdfsa

In [ ]:
import gen3schemadev as g
# rs = g.utils.resolve_schema(schema_path="../tests/gen3_schema/testing/schema_dev_pass.json")
rs = g.utils.resolve_schema(schema_path="../tests/gen3_schema/examples/json/schema_dev.json")

In [ ]:
rs

In [ ]:
g.utils.write_json(rs, "../tests/gen3_schema/examples/json/schema_dev_resolved.json")

In [ ]:
type(rs)

In [ ]:
g.utils.bundled_schema_to_list_dict(rs)